# **Librerie da installare per il db**

In [ ]:
import pandas as pd
import sqlite3
import requests
from io import StringIO

### **Codice per creare il db partendo da url o da file**

In [ ]:
# Definisco l'URL dei file che voglio recuperare
url_importanza_economica_csv = 'http://www.datiopen.it/export/csv/Importanza-economica-del-settore-della-pesca-per-regione.csv'

In [ ]:
# # Utilizzo la libreria requests per ricavarmi il contenuto dei file
response_importanza_economica_csv = requests.get(url_importanza_economica_csv)

In [ ]:
 # Se la richiesta non è andata a buon fine genero un'eccezione
response_importanza_economica_csv.raise_for_status()

In [ ]:
# Se la richiesta è andata a buon fine utilizzo il module di pandas per processare i dati
importanza_economica = pd.read_csv(StringIO(response_importanza_economica_csv.text), sep=';')

In [ ]:
# Patch per il nostro db
db_path = 'cartella_database/cartella_sqlite/pesca.db'

In [ ]:
# Connessione al db
conn = sqlite3.connect(db_path)

In [ ]:
# Creazione tabelle
importanza_economica.to_sql('importanza_economica', conn, index=False, if_exists='replace')

In [ ]:
# Chiusura connessione
conn.close()

### **Codice per le API**

In [1]:
%pip install pandas
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 64.9/64.9 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 99.9/99.9 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 66.8/66.8 kB ? eta 0:00:00
     -------------------------------------- 121.4/121.4 kB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 163.0/163.0 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#from fastapi import FastAPI, HTTPException, Response
#from pydantic import BaseModel, Field
#import matplotlib.pyplot as plt
import sqlite3
from sqlite3 import Error
from io import BytesIO
import pandas as pd
from typing import Optional
import requests
from io import StringIO

In [ ]:
app = FastAPI()

In [ ]:
# Funzione che crea la connessione con il db
def create_connection():
    try:
        conn = sqlite3.connect('cartella_database/cartella_sqlite/pesca.db')

        return conn
    except Error as e:
        print(e)
        return None

In [ ]:
@app.get("/get_media_percentuale_valore_aggiunto_pesca_piscicoltura/")
async def get_media_percentuale_valore_aggiunto_pesca_piscicoltura(year_a: Optional[int] = None,
                                                                   year_b: Optional[int] = None,
                                                                   plot_graph: Optional[bool] = False):
    # Controllo sugli anni
    if year_a is not None and year_b is not None:
        if year_a >= year_b:
            raise HTTPException(status_code=400, detail="Anno A deve essere più piccolo di Anno B")
        
    # Query da utilizzare
    conn = sqlite3.connect('cartella_database/cartella_sqlite/pesca.db')
    query = """
    SELECT *
    FROM importanza_economica
    """

    df = pd.read_sql_query(query, conn)


    # Definizione delle aree geografiche
    nord_ovest = ["Valle d'Aosta", "Piemonte", "Liguria", "Lombardia"]
    nord_est = ["Trentino-Alto Adige", "Veneto", "Friuli-Venezia Giulia", "Emilia-Romagna"]
    centro = ["Toscana", "Umbria", "Marche", "Lazio", "Abruzzo"]
    sud = ["Molise", "Campania", "Puglia", "Basilicata", "Calabria"]
    isole = ["Sicilia", "Sardegna"]

    # Lista delle aree
    aree = [nord_ovest, nord_est, centro, sud, isole]
    nomi_aree = ['Nord-Ovest', 'Nord-Est', 'Centro', 'Sud', 'Isole']


    # Inizializza una lista di dizionari per i risultati
    results = []
    for anno in df['Anno'].unique():
        # Inizializza il dizionario per salvare le medie percentuali per l'anno corrente
        medie_percentuali_anno = {'Anno': anno}
        
        # Itera per ogni area geografica
        for area, nome_area in zip(aree, nomi_aree):
            # Seleziona le righe del DataFrame corrispondenti all'anno e all'area geografica
            subset = df[(df['Anno'] == anno) & (df['Regione'].isin(area))]
            
            # Calcola la media percentuale del valore aggiunto della pesca e piscicoltura per l'area e l'anno correnti
            media_percentuale = subset['Percentuale_valore_aggiunto_pesca-piscicoltura-servizi'].mean()
            
            # Salva la media percentuale nel dizionario
            medie_percentuali_anno[nome_area] = media_percentuale
        
            # Crea un dizionario per il risultato corrente
            result = {
                'Anno': anno,
                'Aree': nome_area,
                'Media percentuale valore aggiunto pesca piscicoltura': media_percentuale
            }
            
            # Aggiungi il dizionario alla lista dei risultati
            results.append(result)

    # Converti la lista di dizionari in un DataFrame
    media_percentuale_valore_aggiunto_pesca_piscicoltura = pd.DataFrame(results)

    # Visualizza il DataFrame dei risultati finali
    display(media_percentuale_valore_aggiunto_pesca_piscicoltura)
    
    # Filtra i dati in base ai parametri year_a e year_b
    if year_a is not None and year_b is not None:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura[
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] >= year_a) &
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] <= year_b)
        ]
    elif year_a is not None and year_b is None:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura[
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] >= year_a)
        ]
    elif year_a is None and year_b is not None:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura[
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] <= year_b)
        ]
    else:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura
        
    # Chiudi la connessione al database
    conn.close()
    
    if plot_graph:
        plt.figure(figsize=(10, 6))
        for area in nomi_aree:
            plt.plot(filtered_data['Anno'], filtered_data[area], marker='o', label=area)
        plt.xlabel('Anno')
        plt.ylabel('Media percentuale valore aggiunto pesca piscicoltura')
        plt.title('Media percentuale valore aggiunto pesca piscicoltura per area geografica')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Salva il grafico in un buffer BytesIO
        buffer = BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)

        # Ritorna il grafico come risposta HTTP
        return Response(content=buffer.getvalue(), media_type='image/png')
    
    return filtered_data 


    

In [5]:
url_importanza_economica_csv = 'http://www.datiopen.it/export/csv/Importanza-economica-del-settore-della-pesca-per-regione.csv'
url_andamento_occupazione_csv = 'http://www.datiopen.it/export/csv/Andamento-occupazione-del-settore-della-pesca-per-regione.csv'
url_produttivita_del_settore_csv = 'http://www.datiopen.it/export/csv/Produttivita-del-settore-della-pesca-per-regione.csv'

response_importanza_economica_csv = requests.get(url_importanza_economica_csv)
response_andamento_occupazione_csv = requests.get(url_andamento_occupazione_csv)
response_produttivita_del_settore_csv = requests.get(url_produttivita_del_settore_csv)

response_importanza_economica_csv.raise_for_status()
response_andamento_occupazione_csv.raise_for_status()
response_produttivita_del_settore_csv.raise_for_status()

importanza_economica = pd.read_csv(StringIO(response_importanza_economica_csv.text), sep=';')
andamento_occupazione = pd.read_csv(StringIO(response_andamento_occupazione_csv.text), sep=';')
produttivita_del_settore = pd.read_csv(StringIO(response_produttivita_del_settore_csv.text), sep=';')

display(importanza_economica)
display(andamento_occupazione)
display(produttivita_del_settore)

,Anno,Regione,Percentuale valore aggiunto pesca-piscicoltura-servizi
0,2000,Piemonte,0.01
1,2000,Valle d'Aosta,0.01
2,2000,Lombardia,0.01
3,2000,Trentino-Alto Adige,0.02
4,2000,Veneto,0.16
...,...,...,...
235,2011,Puglia,0.40
236,2011,Basilicata,0.01
237,2011,Calabria,0.13
238,2011,Sicilia,0.36


,Anno,Regione,Variazione percentuale unità di lavoro della pesca
0,2001,Piemonte,11.89
1,2001,Valle d'Aosta,200.00
2,2001,Lombardia,162.04
3,2001,Trentino-Alto Adige,-0.67
4,2001,Veneto,-2.67
...,...,...,...
235,2012,Puglia,-1.26
236,2012,Basilicata,-17.24
237,2012,Calabria,-12.13
238,2012,Sicilia,-0.79


,Anno,Regione,Produttività in migliaia di euro
0,2000,Piemonte,59.2180
1,2000,Valle d'Aosta,372.0144
2,2000,Lombardia,119.4304
3,2000,Trentino-Alto Adige,43.3656
4,2000,Veneto,24.3476
...,...,...,...
235,2011,Puglia,27.0511
236,2011,Basilicata,48.3853
237,2011,Calabria,27.9066
238,2011,Sicilia,12.7831
